# Procesado de observaciones

Cargar archivos

In [ ]:
import geopandas as gpd
import pandas as pd
import os
from shapely.geometry import Point

gdf_peninsula = gpd.read_file('Mallas_cuadrículas/Mallas Canarias/Malla10km_p.shp')
gdf_canarias = gpd.read_file('Mallas_cuadrículas/Mallas Canarias/Malla10km_c.shp')
df = pd.read_csv('Datos iNaturalist/observations_selection.csv')

Pasar de coordenadas a geometría de puntos

In [ ]:
# Crear geometría de puntos a partir de latitud y longitud
df["geometry"] = df.apply(lambda row: Point(row["decimalLongitude"], row["decimalLatitude"]), axis=1)

# Convertir DataFrame en GeoDataFrame
gdf_points = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")  # CRS WGS84

Transformar los sistemas de coordenadas

- Las coordenadas en el CSV están en WGS84 (EPSG:4326)
- Las cuadrículas SHP están en UTM (Husos 28-32, según la tabla).
- Necesitamos asegurarnos de que ambos usan el mismo sistema de coordenadas (CRS).

In [21]:
# Antes de cambiar sistemas de coordenadas
print("Points crs:" + str(gdf_points.crs))
print("Canarias crs:" + str(gdf_canarias.crs))
print("Peninsula crs:" + str(gdf_peninsula.crs))

# Usar CRS de EPSG:4326
gdf_points = gdf_points.to_crs(epsg=4326)
gdf_peninsula = gdf_peninsula.to_crs(epsg=4326)
gdf_canarias = gdf_canarias.to_crs(epsg=4326)

# Después
print("Points crs:" + str(gdf_points.crs))
print("Canarias crs:" + str(gdf_canarias.crs))
print("Peninsula crs:" + str(gdf_peninsula.crs))


Points crs:EPSG:4326
Canarias crs:EPSG:25830
Peninsula crs:EPSG:32628
Points crs:EPSG:4326
Canarias crs:EPSG:4326
Peninsula crs:EPSG:4326


Realizar unión espacial para asignar cada punto a una cuadrícula de la malla

In [ ]:
# Unión espacial (spatial join) de puntos con cuadrículas de la Península.
# how="left" hace que se incluyan todos los puntos y solo agrega información de la cuadrícula si hay intersección
# predicate="within" solo asigna un punto a una cuadrícula si está dentro de ella.
gdf_result_peninsula = gpd.sjoin(gdf_points, gdf_peninsula, how="left", predicate="within")
gdf_result_canarias = gpd.sjoin(gdf_points, gdf_canarias, how="left", predicate="within")

# Unir los resultados (usando combine_first para dar preferencia a la Península)
gdf_result = gdf_result_peninsula.combine_first(gdf_result_canarias)

# Unir resultados (primero Península, luego Canarias para puntos no asignados)
# Toma valores de gdf_result_peninsula primero y, si hay valores NaN (es decir, si un punto no fue asignado a la Península), usa los de gdf_result_canarias.
gdf_result = gdf_result_peninsula.combine_first(gdf_result_canarias)


Guardar resultados

In [ ]:
# Guardar el CSV con la nueva columna de códigos de malla
df['malla_codigo'] = gdf_result['CUADRICULA']
print(df.columns)

Index(['id', 'occurrenceID', 'modified', 'informationWithheld', 'references',
       'occurrenceRemarks', 'recordedBy', 'recordedByID', 'identifiedBy',
       'identifiedByID', 'eventDate', 'eventTime', 'verbatimEventDate',
       'verbatimLocality', 'decimalLatitude', 'decimalLongitude',
       'coordinateUncertaintyInMeters', 'countryCode', 'stateProvince',
       'identificationID', 'dateIdentified', 'identificationRemarks',
       'taxonID', 'scientificName', 'taxonRank', 'kingdom', 'phylum', 'class',
       'order', 'family', 'genus', 'license', 'rightsHolder',
       'inaturalistLogin', 'publishingCountry', 'projectId', 'sex',
       'lifeStage', 'reproductiveCondition', 'vitality', 'dynamicProperties',
       'geometry', 'malla_codigo'],
      dtype='object')


### Eliminar columnas que no son relevantes para el caso de uso

In [ ]:
df = df[['id', 'occurrenceID', 'references', 'recordedBy', 'recordedByID',
       'eventDate', 'decimalLatitude', 'decimalLongitude', 'scientificName',
       'taxonRank', 'license', 'rightsHolder', 'sex', 'malla_codigo']].copy()

### Asociar idtaxon a las observaciones

In [ ]:
df_taxonomia = pd.read_excel('../EIDOS_taxonomia.xlsx')

# Quitar parentesis y palabras que empiezan por mayuscula y numeros
df_taxonomia['lowered_name'] = df_taxonomia['name'].str.replace(r'[()&]', '', regex=True).apply(
    lambda x: x[0].lower() + ' '.join([w for w in x[1:].split() if w == w.lower() and not w.isdigit()])
)

In [ ]:
def find_best_match(species):
    row_words = species.replace('(', '').replace(')', '').lower().split()
    if len(row_words) < 2:
        return None # No hay suficientes palabras para buscar
    # Filtrar candidatos
    candidates = df_taxonomia[df_taxonomia['lowered_name'].str.contains(' '.join(row_words[:2]))]
    # Si hay una categoría taxonómica, refinar los candidatos
    if len(row_words) > 3 and row_words[2] in ['f.', 'subsp.', 'var.']:
        candidates = candidates[candidates['lowered_name'].str.contains(f' {row_words[3]} ')]
    else:
        candidates = candidates[~candidates['lowered_name'].str.contains(r'\b(?: f\. | subsp\. | var\. )\b', regex=True)]
    if candidates.empty:
        return None
    # Calcular las puntuaciones de coincidencia
    row_words_set = set(row_words)
    candidates['score'] = candidates['lowered_name'].apply(lambda c: 
        0.9 * len(set(c.split()) & row_words_set) / len(row_words) +
        0.1 * len(set(c.split()) & row_words_set) / len(c.split())
    )
    # Obtener el mejor candidato
    best_match = candidates.loc[candidates['score'].idxmax()]
    return best_match['taxonid']

species_id = {species: find_best_match(species) for species in set(df["scientificName"].to_list())}
df['idtaxon'] = df["scientificName"].apply(lambda x: species_id[x])

In [ ]:
df.to_csv("Datos iNaturalist/observations_processed.csv", index=False)

In [ ]:
os.remove('Datos iNaturalist/observations_selection.csv')

# Procesado de media

In [ ]:
import pandas as pd
import os

In [ ]:
df = pd.read_csv('Datos iNaturalist/media_selection.csv')
df.columns

Index(['id', 'type', 'format', 'identifier', 'references', 'created',
       'creator', 'publisher', 'license', 'rightsHolder', 'catalogNumber'],
      dtype='object')


In [ ]:
df = df.loc[~df["type"].isin(["Sound"])]
valores_a_eliminar_formato = ["audio/mpeg", "application/octet-stream", "audio/x-wav", "audio/mp4", "image/gif"]
df = df.loc[~df["format"].isin(valores_a_eliminar_formato)]
df.drop(['publisher', 'catalogNumber'], axis=1)
df.columns

El DataFrame tiene 364612 filas.
El DataFrame tiene 364169 filas.
Index(['id', 'type', 'format', 'identifier', 'references', 'created',
       'creator', 'publisher', 'license', 'rightsHolder', 'catalogNumber'],
      dtype='object')
DataFrame size: 275.36 MB


In [ ]:
df.to_csv("Datos iNaturalist/media_processed.csv", index=False)

In [ ]:
os.remove("Datos iNaturalist/media_selection.csv")